### Pre-Trained Neural Networks Demo
**POLI_SCI 490: Machine Learning in Political Science**
*Code adapted from Nora Webb Williams and Hyein Ko*

**Source Data: Tensorflow Flower Data**

*   *tensorfolow* is used to build, train, and run deep learning models
*   *MobileNetV2* and *ResNet50* are pre-treained image classification models for various computer vision tasks
*   *layers, models* help define and structure custom deep learning models
*   *PIL (Python Image Library)* is used to open, resize, and process image
*   *numpy* helps handle image data as arrays for numerical computation
*   *random* helps generate random numbers of make random choices



In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from sklearn.utils import class_weight
from PIL import Image
import numpy as np
import random

/Users/gustavodiaz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# We will use flower data from tensorflow!
# "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"


In [5]:
# Set seed
my_seed = 112358
random.seed(my_seed)

import os

dirname = os.getcwd()


# Set file path
flower_path = dirname + "/flower_photos/"

In [7]:
# Set image and batch size
# - batch size is the number of training sampled processed at once.
#   we use batches because the full dataset may not fit in memory.
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Set training data and validation data
train_data = tf.keras.utils.image_dataset_from_directory(
    flower_path,
    validation_split = 0.2, #using 80% data as training, 20% as validation
    subset = "training",
    seed = my_seed,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    label_mode="categorical",
    shuffle = True)

val_data = tf.keras.utils.image_dataset_from_directory(
    flower_path,
    validation_split = 0.2,
    subset = "validation",
    seed = my_seed,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    label_mode="categorical")

# Check classes in the dataset
class_names = train_data.class_names
num_classes = len(class_names)

print(f"Number of classes: {num_classes}")
print("List of class:", class_names)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
Number of classes: 5
List of class: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [8]:
# Check whether there is asymmetry in dataset
for class_name in class_names:
    print(f"{class_name}: {len(os.listdir(os.path.join(flower_path, class_name)))}")

daisy: 633
dandelion: 898
roses: 641
sunflowers: 699
tulips: 799


In [9]:
# Compute class weights
y_train = train_data.map(lambda x, y: y).unbatch()
y_train = np.array([np.argmax(label.numpy()) for label in y_train])
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

Class weights: {0: np.float64(1.1673956262425447), 1: np.float64(0.8121715076071923), 2: np.float64(1.1184761904761904), 3: np.float64(1.0794117647058823), 4: np.float64(0.9160686427457099)}


2026-02-12 13:30:23.013509: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
# Use the based model from the pre-trained model MobileNetV2
# Note: You can also try with ResNet50 or other pre-trained models!
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,),
                         include_top=False,
                         weights='imagenet')

# Free the base model
base_model.trainable = False

# Build the full model
model = models.Sequential([
    tf.keras.Input(shape=(224, 224, 3)),
    tf.keras.layers.Rescaling(1./255),
    base_model,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Train the model
# - epoch is one full pass through the entire training dataset
#   one pass is usually not enough for the model to learn well
#   but too many can lead to overfitting (the model will memorize the data)
# - if you have 100 images and train for 2 epoches,
#   the model will see all 100 images 2 times during training

EPOCHS = 2
history = model.fit(train_data,
                    validation_data = val_data,
                    epochs = EPOCHS,
                    class_weight = class_weights_dict)

Epoch 1/2
92/92 ━━━━━━━━━━━━━━━━━━━━ 32s 309ms/step - accuracy: 0.4309 - loss: 1.3770 - val_accuracy: 0.5817 - val_loss: 1.0395
Epoch 2/2
92/92 ━━━━━━━━━━━━━━━━━━━━ 32s 343ms/step - accuracy: 0.8019 - loss: 0.6006 - val_accuracy: 0.7602 - val_loss: 0.6851


Let's Fine-tune!
- Slightly retrain a pretrained model on our specific dataset to improve performance

In [13]:
# Unfreeze the top layers
# - set the last few layers of a pretrained model to be trainable again
base_model.trainable = True

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

# Complie the model using a much lower training rate
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [14]:
# Train the model
epochs = 2
history_fine = model.fit(train_data,
                         epochs = epochs,
                         validation_data = val_data,
                         class_weight = class_weights_dict)

Epoch 1/2
92/92 ━━━━━━━━━━━━━━━━━━━━ 34s 327ms/step - accuracy: 0.8728 - loss: 0.4145 - val_accuracy: 0.8120 - val_loss: 0.5290
Epoch 2/2
92/92 ━━━━━━━━━━━━━━━━━━━━ 30s 331ms/step - accuracy: 0.9103 - loss: 0.2850 - val_accuracy: 0.8406 - val_loss: 0.4481


In [15]:
# Save the model
model_path = dirname + "/flower_model.keras"
model.save(model_path)

Let's predict with a new flower image!

In [16]:
# Load the saved model
# - batch_size 64, train_epochs: 10, fine_tune_epochs: 5
model = tf.keras.models.load_model(model_path)


In [17]:

# Provide the image
img_path = dirname + "/rose.png"

# Load and preprocess image
img = Image.open(img_path).convert("RGB")
img = img.resize(IMG_SIZE)
img_array = np.array(img)
img_array = np.expand_dims(img_array, axis=0)
img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)

# Run prediction
pred = model.predict(img_tensor)
predicted_label = class_names[np.argmax(pred)]

# Show result
img.show()
print(f"Predicted flower: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
Predicted flower: roses
